In [4]:
# Import the required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [5]:
# Load the dataset

df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Basic Pre-Processing: Drop irrelevant variables

data = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


Geography and Gender are categorical variables and we need to apply encoding to it

In [8]:
# Encoding Gender with Label Encoder

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# For geography, as there are more than 2 categories, we use One hot encoding instead of Label Encoding

from sklearn.preprocessing import OneHotEncoder
onehotencoder_geo = OneHotEncoder()
geo_encoder = onehotencoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [10]:
onehotencoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehotencoder_geo.get_feature_names_out(['Geography']))

In [12]:
# Combined dataframe

data = pd.concat([data.drop('Geography', axis= 1), geo_encoded_df], axis=1)

In [13]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [14]:
# Creating Pickle Files To Dump

with open('label_encoded_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehotencoder_geo, file)

In [15]:
# Divide the dataset into Independent and Dependent Features

X = data.drop(columns=['Exited'], axis=1)
y = data['Exited']

# Train Test Split

X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)


# Feature Standardization

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
# Save Scaler

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN IMPLEMENTATION


In [1]:
# Importing the required Libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [17]:
# Creating the Basic Model

model = Sequential([
    Dense(64, activation= 'relu', input_shape = (X_train.shape[1],)),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [19]:
# Model Summary

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
import tensorflow

In [21]:
# Compile the model to add optimizer, loss, and metrics

opt = tensorflow.keras.optimizers.Adam(learning_rate= 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [24]:
# setup tensorboard to capture training logs

logs_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=logs_dir, histogram_freq=1)

In [31]:
# Setting up Early Stopping

early_stopping_callback = EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

In [32]:
# Model Training

history = model.fit(X_train, 
          y_train, 
          validation_data= (X_test, y_test), 
          epochs=100, 
          callbacks=[early_stopping_callback,tensorboard_callback])

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3287 - accuracy: 0.8684 - val_loss: 0.3512 - val_accuracy: 0.8525
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3255 - accuracy: 0.8640 - val_loss: 0.3551 - val_accuracy: 0.8565
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3278 - accuracy: 0.8650 - val_loss: 0.3468 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3219 - accuracy: 0.8664 - val_loss: 0.3530 - val_accuracy: 0.8595
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3218 - accuracy: 0.8670 - val_loss: 0.3533 - val_accuracy: 0.8605
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3173 - accuracy: 0.8666 - val_loss: 0.3647 - val_accuracy: 0.8550
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3134 - accuracy: 0.8700 - val_loss: 0.3579 - val_accuracy: 0.8575

In [33]:
# Save model

model.save('model.h5')

c:\Users\HP\Desktop\DL -Customer Churn Prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
# Load Tensorboard

%load_ext tensorboard

In [36]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 30504), started 1 day, 0:31:03 ago. (Use '!kill 30504' to kill it.)